# This notebook will be used in Coursera Capstone Project

## Import libraries

In [35]:
import pandas as pd # for data analysis
import numpy as np # for data analysis, matrix, vector etc.
import requests # library to handle requests
from bs4 import BeautifulSoup # for webpage scrapping


In [ ]:
# print("Hello Capstone Project Course!")

## Scraping Toronto ZipCode/Neighborhood Data from Wikipedia and Data Wrangling

In [36]:
# get data from Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers= {'User-Agent':'Mozilla/5.0'}
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
wikitable = soup.find('table', attrs={'class':'wikitable sortable'})
response.status_code # status_code 200 means content was downloaded correctly

200

In [39]:
# Identify columns and retrieve headers:
headers=wikitable.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")
headers

['Postcode', 'Borough', 'Neighbourhood']

In [41]:
# Create lists for each column,
A=[]
B=[]
C=[]

for row in wikitable.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [42]:
# Clean up each list by dropping \n
A= [w.replace('\n', '') for w in A]
B= [w.replace('\n', '') for w in B]
C= [w.replace('\n', '') for w in C]

In [46]:
# Create dataframe using each list
import pandas as pd
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C

In [49]:
# Filter out rows where Borough is not assigned
df2 = df.loc[df['Borough']!= 'Not assigned']

In [51]:
# copy Borough value to Neighborhood if NaN:
df2.Neighbourhood.fillna(df2.Borough, inplace=True)
df2

/Users/serifustun/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [53]:
# merge multiple neighbourhoods with the same Postcode
df3 = df2.groupby('Postcode').aggregate(lambda tdf: tdf.unique().tolist())
df3.reset_index(level=0, inplace=True)

In [55]:
df3.head(11)

,Postcode,Borough,Neighbourhood
0,M1B,[Scarborough],"[Rouge, Malvern]"
1,M1C,[Scarborough],"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,[Scarborough],"[Guildwood, Morningside, West Hill]"
3,M1G,[Scarborough],[Woburn]
4,M1H,[Scarborough],[Cedarbrae]
5,M1J,[Scarborough],[Scarborough Village]
6,M1K,[Scarborough],"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,[Scarborough],"[Clairlea, Golden Mile, Oakridge]"
8,M1M,[Scarborough],"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,[Scarborough],"[Birch Cliff, Cliffside West]"


In [56]:
df3.shape

(103, 3)